In [1]:
from dask.distributed import Client
import dask.dataframe as dd
import numpy as np

In [2]:
client = Client()

In [3]:
client

Client Scheduler: tcp://127.0.0.1:36511 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.79 GB


In [4]:
client.restart()

Client Scheduler: tcp://127.0.0.1:36511 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.79 GB


In [4]:
%%time

dtypes = {
    'reviewerID': np.str,
    'asin': np.str,
    'reviewerName': np.str,
    'helpful': np.object,
    'reviewTest': np.str,
    'overall': np.float64,
    'summary': np.str,
    'unixReviewTime': np.float64,
    'reviewTime': np.str
}

df = dd.read_csv('user_reviews.csv', dtype=dtypes)


CPU times: user 26.3 ms, sys: 4.53 ms, total: 30.8 ms
Wall time: 27.3 ms


In [6]:
%%time

#df_1 = df.groupby('asin')['overall'].count(split_out=4).reset_index()
df.head(5)

CPU times: user 59.7 ms, sys: 6.41 ms, total: 66.1 ms
Wall time: 1.32 s


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2T0RJ91B0PQ03,B0016CRVLW,Gerald DeWitt,"[0, 0]",Beware! This is NOT the original single versi...,1.0,Poor Quality Alternate Take,1.400630e+09,"05 21, 2014"
1,A3TYW0XA8HSGWB,B00EKR5S0Q,Linda E. Larson,"[0, 0]",This is my new most favorite k-cup coffee. I c...,5.0,Vanilla Starbucks K-cups,1.398557e+09,"04 27, 2014"
2,A2CME0TQU2IVVB,B001AUPJVO,L5Momma,"[1, 1]",This headset is great! It worked in our 2007 ...,5.0,Awesome!,1.355875e+09,"12 19, 2012"
3,A2E5IDLX7R388S,B000055Y57,Jeff Andersen,"[0, 0]",Scofield is one of my favorite musicians and i...,5.0,Straight ahead Jazz with the Scofield twist,1.402358e+09,"06 10, 2014"
4,A3CIEMYUGV6ZMR,0545265355,Adroit,"[0, 0]","Wonderful book! I cried, well teared up at a f...",5.0,Greatest Book Ever!!!,1.334102e+09,"04 11, 2012"


In [5]:
%%time

# Subset of df
test_df = df[['reviewerID', 'overall', 'reviewTime', 'helpful']]
#test_df = test_df.loc[:20000]

CPU times: user 1.88 ms, sys: 0 ns, total: 1.88 ms
Wall time: 1.76 ms


In [6]:
%%time

# year of review
# Try using unixReviewTime, might be faster
#test_df['reviewYear'] = test_df['reviewTime'].apply(lambda x: int(x[-4:]), meta=np.int)
test_df['reviewYear'] = test_df['reviewTime'].apply(lambda x: int(x[-4:]), meta=np.int)

CPU times: user 7.27 ms, sys: 0 ns, total: 7.27 ms
Wall time: 6.68 ms


In [7]:
%%time
#instead of apply, use map_partitions

# string list to list
test_df['helpful votes'] = test_df['helpful'].apply(lambda x: x.strip('][').split(', ')[0], meta=np.int)

CPU times: user 7.28 ms, sys: 148 µs, total: 7.43 ms
Wall time: 6.68 ms


In [8]:
%%time

test_df['total votes'] = test_df['helpful'].apply(lambda x: x.strip('][').split(', ')[1], meta=np.int)

CPU times: user 3.98 ms, sys: 4.33 ms, total: 8.31 ms
Wall time: 7.24 ms


In [9]:
%%time

number_products_rated = test_df.groupby('reviewerID').count(split_out=4)

CPU times: user 10.7 ms, sys: 0 ns, total: 10.7 ms
Wall time: 9.61 ms


In [10]:
%%time

avg_ratings = test_df.groupby('reviewerID')['overall'].mean(split_out=4)

CPU times: user 19.8 ms, sys: 0 ns, total: 19.8 ms
Wall time: 17.9 ms


In [11]:
%%time

reviewing_since = test_df.groupby('reviewerID')['reviewYear'].min(split_out=4)

CPU times: user 11 ms, sys: 0 ns, total: 11 ms
Wall time: 10.1 ms


In [12]:
%%time

helpful_votes = test_df.groupby('reviewerID')['helpful votes'].sum(split_out=4)

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 9.14 ms


In [13]:
%%time
total_votes = test_df.groupby('reviewerID')['total votes'].sum(split_out=4)

CPU times: user 72.3 ms, sys: 0 ns, total: 72.3 ms
Wall time: 69.5 ms


In [14]:
%%time
reviewerID = number_products_rated.index

CPU times: user 991 µs, sys: 0 ns, total: 991 µs
Wall time: 986 µs


In [ ]:
%%time
result_df = dd.concat([number_products_rated, avg_ratings, reviewing_since, helpful_votes, total_votes], axis=1,
                      ignore_unknown_divisions=True)
result_df.head(10)

In [20]:
result_df.columns = ['number_products_rated', 'avg_ratings', 'reviewing_since', 'helpful_votes', 'total_votes']


In [21]:
result_df = result_df.reset_index()

In [22]:
%% time
result_df.head(10)

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting


KilledWorker: ("('read-csv-assign-15e9aacc986cf71fb3842b45e3903613', 307)", <Worker 'tcp://127.0.0.1:34949', name: 0, memory: 0, processing: 336>)

In [1]:
from dask.distributed import Client
import dask.dataframe as dd
import numpy as np

def PA0(user_reviews_csv):
    client = Client()
    client = client.restart()
    
    dtypes = {
    'reviewerID': np.str,
    'asin': np.str,
    'reviewerName': np.str,
    'helpful': np.object,
    'reviewTest': np.str,
    'overall': np.float64,
    'summary': np.str,
    'unixReviewTime': np.float64,
    'reviewTime': np.str}

    df = dd.read_csv(user_reviews_csv, dtype=dtypes)
    
    test_df = df[['reviewerID', 'overall', 'reviewTime', 'helpful']]
    
    test_df['reviewYear'] = test_df['reviewTime'].apply(lambda x: int(x[-4:]), meta=np.int)
    test_df['helpful votes'] = test_df['helpful'].apply(lambda x: x.strip('][').split(', ')[0], meta=np.int)
    test_df['total votes'] = test_df['helpful'].apply(lambda x: x.strip('][').split(', ')[1], meta=np.int)
    
    number_products_rated = test_df.groupby('reviewerID').count(split_out=4)
    avg_ratings = test_df.groupby('reviewerID')['overall'].mean(split_out=4)
    reviewing_since = test_df.groupby('reviewerID')['reviewYear'].min(split_out=4)
    helpful_votes = test_df.groupby('reviewerID')['helpful votes'].sum(split_out=4)
    total_votes = test_df.groupby('reviewerID')['total votes'].sum(split_out=4)
    
    result_df = dd.concat([number_products_rated, avg_ratings, reviewing_since, helpful_votes, total_votes], axis=1,
                      ignore_unknown_divisions=True)
    
    result_df.columns = ['number_products_rated', 'avg_ratings', 'reviewing_since', 'helpful_votes', 'total_votes']
    
    result_df = result_df.reset_index()
    
    submit = result_df.describe().compute().round(2)    
    with open('results_PA0.json', 'w') as outfile: json.dump(json.loads(submit.to_json()), outfile)

In [2]:
PA0('user_reviews.csv')

ValueError: Length mismatch: Expected axis has 10 elements, new values have 5 elements